<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/projects/next_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Create model from scratch

In [1]:
import numpy as np
import requests

file_url = 'https://raw.githubusercontent.com/krahul2024/machine-learning/main/projects/Text-Datasets/paul_graham_essays.txt'
content = requests.get(file_url).text
len(content)

3023219

In [2]:
vocab = sorted(list(set(content)))
vocab_size, total_chars = len(vocab), len(content)
vocab_size

106

In [3]:
#@title Imports related to the model
import tensorflow as tf
from tensorflow import keras

In [4]:
char_index = dict((c, i) for i, c in enumerate(vocab))
char_index

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 '*': 11,
 '+': 12,
 ',': 13,
 '-': 14,
 '.': 15,
 '/': 16,
 '0': 17,
 '1': 18,
 '2': 19,
 '3': 20,
 '4': 21,
 '5': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 ':': 27,
 ';': 28,
 '<': 29,
 '=': 30,
 '>': 31,
 '?': 32,
 '@': 33,
 'A': 34,
 'B': 35,
 'C': 36,
 'D': 37,
 'E': 38,
 'F': 39,
 'G': 40,
 'H': 41,
 'I': 42,
 'J': 43,
 'K': 44,
 'L': 45,
 'M': 46,
 'N': 47,
 'O': 48,
 'P': 49,
 'Q': 50,
 'R': 51,
 'S': 52,
 'T': 53,
 'U': 54,
 'V': 55,
 'W': 56,
 'X': 57,
 'Y': 58,
 'Z': 59,
 '[': 60,
 ']': 61,
 '^': 62,
 '_': 63,
 '`': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93,
 '\x92': 94,
 '\x96': 95,
 '\x97': 96,
 '\xa0': 97,
 '²': 98,
 'à': 9

In [5]:
input_size = 40
steps = 3

x , y = [], []
for i in range(0, total_chars-input_size, steps):
    x.append(content[i:i+input_size])
    y.append(content[i+input_size])

for i in range(10):
    print(f"{x[i]} -> {y[i]}")

print(f"input-size : {len(x)}, output-size : {len(y)}")

  Anyone can see they're not the same by ->  
nyone can see they're not the same by th -> e
ne can see they're not the same by the n -> u
can see they're not the same by the numb -> e
 see they're not the same by the number  -> o
e they're not the same by the number of  -> p
hey're not the same by the number of peo -> p
're not the same by the number of people ->  
 not the same by the number of people wh -> o
t the same by the number of people who a -> r
input-size : 1007727, output-size : 1007727


In [6]:
X = np.zeros(
    (
         len(x),
         input_size,
         len(vocab)
    ),
    dtype =bool
)

Y = np.zeros(
    (
        len(x),
        vocab_size
    )
)

for i, seq in enumerate(x):
  for j, char in enumerate(seq):
    X[i, j, char_index[char]] = 1
    Y[i, char_index[y[i]]] = 1

X.shape, Y.shape

((1007727, 40, 106), (1007727, 106))

In [7]:
X[0][0], Y[0]

(array([False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

In [9]:
# create the model
model = keras.models.Sequential([
    keras.layers.LSTM(128, input_shape = (input_size, vocab_size)),
    keras.layers.Dense(
        vocab_size, activation = 'softmax'
    )
])

# compile the model
model.compile(
    optimizer = 'adam',
    metrics = ['accuracy'],
    loss = 'categorical_crossentropy'
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               120320    
                                                                 
 dense_1 (Dense)             (None, 106)               13674     
                                                                 
Total params: 133994 (523.41 KB)
Trainable params: 133994 (523.41 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# function which takes model and various parameters to train the model on given data
def train(model_, input_data, output_data, batch_size = 128, validation_split = 0.05, verbose = 1, shuffle = True, epochs = 20):
  training_history = model_.fit(
      input_data, output_data, batch_size = batch_size,
      validation_split = validation_split, verbose = verbose,
      shuffle = shuffle, epochs = epochs
  ).history

  return model_, training_history

In [11]:
# Now we will divide the data into multiple parts and train each part seperately once training for one part is finished
# This is to tackle the colab ram restrictions

train_inputs = np.array_split(X, 10)
train_outputs = np.array_split(Y, 10)

train_inputs[0].shape, train_outputs[0].shape

((100773, 40, 106), (100773, 106))

In [12]:
model, history = train(model, train_inputs[0], train_outputs[0], epochs = 40)

Epoch 1/40
748/748 [==============================] - 14s 10ms/step - loss: 2.8029 - accuracy: 0.2459 - val_loss: 2.5263 - val_accuracy: 0.3207
Epoch 2/40
748/748 [==============================] - 5s 7ms/step - loss: 2.3970 - accuracy: 0.3303 - val_loss: 2.3722 - val_accuracy: 0.3461
Epoch 3/40
748/748 [==============================] - 6s 8ms/step - loss: 2.2662 - accuracy: 0.3588 - val_loss: 2.2822 - val_accuracy: 0.3588
Epoch 4/40
748/748 [==============================] - 5s 7ms/step - loss: 2.1848 - accuracy: 0.3750 - val_loss: 2.2184 - val_accuracy: 0.3697
Epoch 5/40
748/748 [==============================] - 5s 6ms/step - loss: 2.1234 - accuracy: 0.3899 - val_loss: 2.1682 - val_accuracy: 0.3844
Epoch 6/40
748/748 [==============================] - 6s 8ms/step - loss: 2.0735 - accuracy: 0.4025 - val_loss: 2.1240 - val_accuracy: 0.3999
Epoch 7/40
748/748 [==============================] - 5s 6ms/step - loss: 2.0308 - accuracy: 0.4135 - val_loss: 2.0988 - val_accuracy: 0.4033
Epoc

In [13]:
model.save('next_char.keras')

### Improve the model

In [ ]:
'''
1. Load the content and model
2. Divide the data into multiple parts
3. Train one part at a time
4. Save the model upto that training
5. Continue training the model for other parts
'''

